In [2]:
import os
import re
import fnmatch
import numpy as np
import cv2 as cv
import tqdm
import concurrent.futures

p_cutoff = 0.7
vote_threshold = 0.5
suffix_long = 49
frame_rate = 50.0
raw_data_dir = r'G:\works\Data_archive\8f_behav\balance_beam\2023May\croped'
output_dir = r'D:\works\Data_analysis\projects\balance_beam\2023May\preprocessed_DLC_data'

In [12]:
def file_name_extraction(filename):
    filename = filename.replace(raw_data_dir+'\\',"")
    filename = filename[:suffix_long*-1]
    return filename

def find_frame_need_knocked(data):
    knocked_frame = []
    for index,frame in enumerate(data[3:]):
        kick_vote = 0 

        for likehood in frame[3::3]:
            if (float(likehood)<p_cutoff):
                kick_vote+=1

        if (kick_vote >= len(frame)//(3/vote_threshold)):
            knocked_frame.append(index)
            
    knocked_frame_csv = [x + 3 for x in knocked_frame]
    return knocked_frame, knocked_frame_csv
            
def csv_process(filename,data,knocked_frame):
    data = np.delete(data, knocked_frame, axis=0)
    output_folder = os.path.join(output_dir, "pose_estimation")
    
    # 創建目錄
    os.makedirs(output_folder, exist_ok=True)
    
    output_name = os.path.join(output_folder, f"{filename}.csv")
    np.savetxt(output_name, data, delimiter=",",fmt='%s')
            
def video_process(filename,knocked_frame): 
    output_name = output_dir + "\\" + filename + ".mp4"
    
    for filename in fnmatch.filter(filenames, filename+'.mp4'):
        raw_video_dir = os.path.join(root, filename)
        
    video = cv.VideoCapture(raw_video_dir)
    total_frames_orginal = int(video.get(cv.CAP_PROP_FRAME_COUNT))
    output_frames_should_be = int(video.get(cv.CAP_PROP_FRAME_COUNT)) - len(knocked_frame)

    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    output = cv.VideoWriter(output_name, fourcc, frame_rate, (int(video.get(3)), int(video.get(4))))
    
    i = 0
    while True:
        ret, frame = video.read()
        if not ret or i >= output_frames_should_be:
            break
        if video.get(cv.CAP_PROP_POS_FRAMES) not in knocked_frame:
            output.write(frame)
            i+=1
        
        
    video.release()
    output.release()

In [4]:
csv_files = []
mp4_files = []
for root, dirnames, filenames in os.walk(raw_data_dir):
    for filename in fnmatch.filter(filenames, '*.csv'):
        csv_files.append(os.path.join(root, filename))
    for filename in fnmatch.filter(filenames, '*.mp4'):
        mp4_files.append(os.path.join(root, filename))
print('total find '+str(len(csv_files))+' csv_files and '+str(len(mp4_files))+' mp4_files')

total find 45 csv_files and 45 mp4_files


In [9]:
# single file test
fill_name = file_name_extraction(csv_files[38])
print(fill_name)
csv = np.genfromtxt(csv_files[38],dtype=None,delimiter=',',encoding='UTF-8')
frame_need_knocked_forvideo, frame_need_knocked_forcsv = find_frame_need_knocked(csv)
csv_process(fill_name,csv,frame_need_knocked_forcsv)
video_process(fill_name,frame_need_knocked_forvideo)

0522_DoB1213E_CTZ_cropped
12144 7215 4929


In [11]:
# All file
def process_csv(csv_dir):
    file_name = file_name_extraction(csv_dir)
    csv = np.genfromtxt(csv_dir,dtype=None,delimiter=',',encoding='UTF-8')
    frame_need_knocked_forvideo, frame_need_knocked_forcsv = find_frame_need_knocked(csv)
    csv_process(file_name,csv,frame_need_knocked_forcsv)
    video_process(file_name,frame_need_knocked_forvideo)

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = [executor.submit(process_csv, csv_dir) for csv_dir in csv_files]
    for f in tqdm.tqdm(concurrent.futures.as_completed(results), total=len(results)):
        pass

0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped
0522_DoB1213E_CTZ_cropped


  0%|          | 0/45 [00:00<?, ?it/s]

9576 6772 2804
9696 6396 3300
10536 7019 3517
10800 6913 3887
11592 7597 3995
12144 7441 4703
15408 12299 3109
15240 9225 6015
15624 8127 7497
15216 6778 8438
17496 11253 6243
16896 9800 7096
17712 9737 7975
16992 8274 8718
17616 9254 8362
18864 10315 8549
17880 7634 10246
20832 9693 11139
21720 11549 10171
23832 10158 13674
26520 10435 16085
26832 11269 15563
26496 10320 16176
27288 11174 16114


  2%|▏         | 1/45 [02:23<1:45:33, 143.93s/it]

0522_DoB1213E_CTZ_cropped


...

 47%|████▋     | 21/45 [06:45<04:54, 12.25s/it]

0522_DoB1213E_CTZ_cropped


 51%|█████     | 23/45 [06:51<02:47,  7.61s/it]

14016 10006 4010


100%|██████████| 45/45 [08:52<00:00, 11.84s/it]


In [ ]:
#debug
video = cv.VideoCapture(r'G:\works\Data_archive\8f_behav\balance_beam\2022Autumn\dlc_analyze_temp\wt5_ns.mp4')
total_frames = int(video.get(cv.CAP_PROP_FRAME_COUNT))
video.release()
print("Total frames in orginal:", total_frames)

print("frame be killed:",len(frame_need_knocked_forvideo))

print("Total frames should:",total_frames-len(frame_need_knocked_forvideo))

video = cv.VideoCapture(r'D:\works\Data_analysis\projects\balance_beam\Balance_Beam_VAME-MX-Feb8-2023\videos\wt5_ns.mp4')
total_frames = int(video.get(cv.CAP_PROP_FRAME_COUNT))
video.release()
print("Total frames after process:", total_frames)

Total frames in orginal: 17422
frame be killed: 12299
Total frames should: 5123
Total frames after process: 0


In [10]:
csv_files

['G:\\works\\Data_archive\\8f_behav\\balance_beam\\2023May\\croped\\0518_DoB0904A_baseline_croppedDLC_resnet50_Balance_BeamMay29shuffle1_150000.csv',
 'G:\\works\\Data_archive\\8f_behav\\balance_beam\\2023May\\croped\\0518_DoB0904B_baseline_croppedDLC_resnet50_Balance_BeamMay29shuffle1_150000.csv',
 'G:\\works\\Data_archive\\8f_behav\\balance_beam\\2023May\\croped\\0518_DoB0904C_baseline_croppedDLC_resnet50_Balance_BeamMay29shuffle1_150000.csv',
 'G:\\works\\Data_archive\\8f_behav\\balance_beam\\2023May\\croped\\0518_DoB0904D_baseline_croppedDLC_resnet50_Balance_BeamMay29shuffle1_150000.csv',
 'G:\\works\\Data_archive\\8f_behav\\balance_beam\\2023May\\croped\\0518_DoB1213A_baseline_croppedDLC_resnet50_Balance_BeamMay29shuffle1_150000.csv',
 'G:\\works\\Data_archive\\8f_behav\\balance_beam\\2023May\\croped\\0518_DoB1213B_baseline_croppedDLC_resnet50_Balance_BeamMay29shuffle1_150000.csv',
 'G:\\works\\Data_archive\\8f_behav\\balance_beam\\2023May\\croped\\0518_DoB1213C_baseline_croppedDL